In [50]:
import pandas as pd
import numpy as np
from pandas import Timedelta
from datetime import timedelta
from sklearn.datasets import load_wine
from sklearn.ensemble import GradientBoostingClassifier

In [51]:
df = pd.read_pickle(r"../../models/data_train/'begin_Bloaters', 'begin_Couplers', 'begin_Object-Orientation Abusers'_3.pkl")
df = df.drop_duplicates()

In [52]:
date = pd.to_timedelta(df['total_time'])
time_hours = date.dt.total_seconds() / 3600

df['hours'] = time_hours

In [53]:
result_time  = []

for i in df['hours']:
    if i > 153.70 :
        result_time.append(0)
    elif (i>=48.54) & (i<=153.70):
        result_time.append(1)
    else:
        i <= 48.54
        result_time.append(2)
df['time_class'] = result_time

In [54]:
X = df.loc[:, ~df.columns.isin([
                                'hours', 'scored' , 'clusters' , 
                                r'[\'begin_Bloaters\', \'begin_Couplers\', \'begin_Object-Orientation Abusers\']_3' ,
                                'total_time' ,
                                'begin_sha', 'end_sha', 'begin_time' , 'begin_key', 'begin_revision', 'end_time', 'end_key', 'end_revision'
                                ])]
y = df['time_class']

In [55]:
x_drop = X.columns[-2]

In [56]:
X.drop(x_drop, axis=1)

,commits,additions,deletions,changed_files,begin_Dispensables,begin_Bloaters,begin_Change Preventers,begin_Couplers,begin_Object-Orientation Abusers,end_Dispensables,end_Bloaters,end_Change Preventers,end_Couplers,end_Object-Orientation Abusers,begin_Bloaters,begin_Couplers,begin_Object-Orientation Abusers,time_class
0,1,10,4,3,500.0,262.0,262.0,304.0,304.0,500.0,261.0,261.0,304.0,693.0,262.0,304.0,304.0,1
1,1,2,1,1,500.0,262.0,262.0,304.0,304.0,501.0,262.0,262.0,304.0,693.0,262.0,304.0,304.0,1
2,1,36,67,9,500.0,262.0,262.0,304.0,304.0,499.0,261.0,261.0,304.0,693.0,262.0,304.0,304.0,2
3,1,21,2,3,500.0,262.0,262.0,304.0,304.0,500.0,261.0,261.0,304.0,693.0,262.0,304.0,304.0,1
4,8,211,212,80,499.0,261.0,261.0,304.0,304.0,500.0,262.0,262.0,304.0,693.0,261.0,304.0,304.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,4,191,40,12,16.0,14.0,14.0,7.0,7.0,16.0,15.0,15.0,7.0,102.0,14.0,7.0,7.0,2
1064,6,470,42,16,14.0,12.0,12.0,8.0,8.0,16.0,14.0,14.0,7.0,89.0,12.0,8.0,8.0,2
1065,2,69,49,9,14.0,12.0,12.0,8.0,8.0,16.0,14.0,14.0,7.0,89.0,12.0,8.0,8.0,2
1066,1,11,1128,27,2.0,3.0,3.0,3.0,3.0,16.0,14.0,14.0,7.0,89.0,3.0,3.0,3.0,2


In [57]:
X_train, X_test = X[:int(0.7 * len(X))], X[int(0.7 * len(X)):]
y_train, y_test = y[:int(0.7 * len(X))], y[int(0.7 * len(X)):]

In [58]:
int(0.7 * len(X))

746

In [59]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
clf = make_pipeline(StandardScaler(), SVC(gamma='auto' , random_state = 2))

In [60]:
from sklearn.model_selection import cross_val_score
cross_val_score(clf , X , y , cv=5 , scoring= "accuracy")

array([0.48130841, 0.99061033, 0.99530516, 0.99061033, 0.88262911])

In [61]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(clf,X,y,cv=5)
y_pred

array([1, 1, 1, ..., 2, 2, 2], dtype=int64)

In [62]:
confusion_matrix(y,y_pred)

array([[132,  26,   0],
       [  3, 188,   3],
       [  0, 109, 605]], dtype=int64)

In [63]:
from sklearn.metrics import precision_score, recall_score, f1_score
print('precision = ', precision_score(y,y_pred ,average='macro'))
print('recall_score = ', recall_score(y,y_pred ,average='macro'))
print('f1_score',f1_score(y,y_pred ,average='macro'))

precision =  0.8516289703015709
recall_score =  0.8839513794991224
f1_score 0.8478588323429886


In [64]:
frame={"y":y,"y_pred": y_pred}
pd.DataFrame(frame).sum()

y         1622
y_pred    1539
dtype: int64

In [65]:
from sklearn.model_selection import GridSearchCV

parameters = {

    'C': [0.01 , 0.1 ,1],
    'gamma' : ['scale' , 'auto'],
    'random_state':[2]

}
svc = SVC()
clf = GridSearchCV(svc , parameters , cv=3 , scoring='f1_macro')
clf.fit(X,y)

GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': [0.01, 0.1, 1], 'gamma': ['scale', 'auto'],
                         'random_state': [2]},
             scoring='f1_macro')

In [66]:
clf.predict(X_test)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2,

In [67]:
clf.best_estimator_

SVC(C=1, random_state=2)

In [68]:
y_pred = clf.predict(X)
y_pred

array([2, 2, 2, ..., 2, 2, 2], dtype=int64)

In [69]:
clf.predict(X_test)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2,

In [70]:
y_pred_clf = clf.best_estimator_.predict(X)
y_pred_clf

array([2, 2, 2, ..., 2, 2, 2], dtype=int64)

In [71]:
from sklearn.metrics import precision_score, recall_score, f1_score
print('precision = ', precision_score(y,y_pred_clf ,average='macro'))
print('recall_score = ', recall_score(y,y_pred_clf,average='macro' ))
print('f1_score',f1_score(y,y_pred_clf,average='macro' ))

precision =  0.759475160734012
recall_score =  0.38506840380242746
f1_score 0.35932140740044066


In [72]:
clf.predict(X_test)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2,

In [73]:
from matplotlib import pyplot as plt
from sklearn import svm
import numpy as np

def f_importances(coef, names):

    names = np.arange(1)
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

features_names = X.columns
svm1 = svm.SVC(kernel='linear',C=1)
svm1.fit(X, y)

SVC(C=1, kernel='linear')

In [74]:
svm1.coef_

array([[ 2.28813947e-03, -7.23466437e-05, -2.12042646e-05,
         1.14487696e-03,  4.40685724e-03, -1.71924374e-03,
        -1.71924374e-03,  2.53235035e-03,  2.53235035e-03,
        -2.87470693e-03, -1.50786320e-03, -1.50786320e-03,
        -7.24990168e-03,  8.98777549e-04, -1.71924374e-03,
         2.53235035e-03,  2.53235035e-03, -7.93852484e-02,
        -2.53724139e+00],
       [ 3.19730339e-06, -1.57093339e-07, -3.75384797e-08,
         1.47698030e-06,  2.77077760e-07, -2.48938008e-07,
        -2.48938008e-07, -3.35555825e-07, -3.35555825e-07,
         5.83348026e-07,  8.64976911e-07,  8.64976911e-07,
         3.81800191e-08, -5.40130372e-07, -2.48938008e-07,
        -3.35555825e-07, -3.35555825e-07,  2.06121859e-04,
        -9.99998314e-01],
       [ 1.22373323e-06, -8.53798809e-10,  5.57020128e-08,
        -1.68316772e-06, -3.91409003e-08, -3.99670057e-08,
        -3.99670057e-08,  3.82085204e-07,  3.82085204e-07,
         2.01372586e-07,  1.28863252e-06,  1.28863252e-06,
    